In [0]:
import random
import math
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, f1_score,confusion_matrix,classification_report

In [0]:
def model_score(x_train, y_train, x_test, y_test, classes = 'multi'):
    
    #x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)
    model = DecisionTreeClassifier(random_state = 101)
    #print(type(model))
    model.fit(x_train,y_train)
    
    test_predictions = model.predict(x_test)
    if classes == 'multi':
        return f1_score(y_test, test_predictions, average = 'macro')
    else:
        return f1_score(y_test, test_predictions)

In [0]:
def obj_func(pos_array,X_train,y_train,X_test, y_test, classes):
    alpha = 0.6
    feature_array = np.zeros_like(pos_array)
    for i in range(len(pos_array)):
        #print(1/(1+math.exp(-pos_array[i])))
        feature_array[i] = 1/(1+math.exp(-pos_array[i]))
    feature_array = [round(i) for i in feature_array]
    feature_index = [i for i in range(len(feature_array)) if feature_array[i] > 0]
    #print(len(feature_index))
    if (len(feature_index)==0):
        return [0,0]
    
    f1_score = model_score(X_train, y_train, X_test, y_test, classes = classes)
    fitness_score = alpha*f1_score + (1-alpha)*(X_train.shape[1]-len(feature_index))/X_train.shape[1]
    return [fitness_score,f1_score]


In [0]:
class solution:
    def __init__(self):
        self.best = 0
        self.bestIndividual=[]
        self.convergence = []
        self.optimizer=""
        self.objfname=""

In [0]:
def WOA(X_train, y_train, X_test, y_test, objf = obj_func, classes = 'multi', lb = -100, ub = 100,
        SearchAgents_no = 30, Max_iter = 50):

    dim = X_train.shape[1]        
    Leader_pos=np.zeros(dim)
    Leader_score=0
    Leader_f1 = 0  
 
    Positions = np.zeros((SearchAgents_no, dim))
    for i in range(dim):
        Positions[:, i] = np.random.uniform(0,1,SearchAgents_no) *(ub-lb)+lb
    
    convergence_curve=np.zeros(Max_iter)
    s=solution()
    
    t=0 
    while t<Max_iter:
        for i in range(0,SearchAgents_no):
            for j in range(dim):        
                Positions[i,j]=np.clip(Positions[i,j], lb, ub)
            
            fitness,f1score=objf(Positions[i,:],X_train,y_train,X_test,y_test, classes = classes)
     
            if fitness>Leader_score: 
                Leader_score=fitness; 
                Leader_pos=Positions[i,:].copy()
                Leader_f1= f1score  
        
        a=2-t*((2)/Max_iter); 

        for i in range(0,SearchAgents_no):
            r1=random.random() 
            r2=random.random() 
            
            A=2*a*r1-a                                            
            C=2*r2                                                
            
            b=1            
            l=random.randint(-1,1)                            
            
            p = random.random() 

            if p<0.5:
                
                if abs(A)>=1:
                    rand_leader_index = math.floor(SearchAgents_no*random.random());
                    X_rand = Positions[rand_leader_index, :]
                    D_random= abs(C*X_rand-Positions[i,:])
                    Positions[i,:]=X_rand-A*D_random      
                    
                elif abs(A)<1:  
                    D_Leader=abs(C*Leader_pos-Positions[i,:])
                    Positions[i,:]=Leader_pos-A*D_Leader    
                        
            elif p>=0.5:  
                distance2Leader=abs(Leader_pos-Positions[i,:])                    
                Positions[i,:]=distance2Leader*math.exp(b*l)*math.cos(l*2*math.pi)+Leader_pos
                
                    
        convergence_curve[t]=Leader_score
        t=t+1
    
    feature_array = np.zeros_like(Leader_pos)
    for i in range(len(Leader_pos)):
        feature_array[i] = 1/(1+math.exp(-Leader_pos[i]))
    feature_array = [round(i) for i in feature_array]
    feature_index = [i for i in range(len(feature_array)) if feature_array[i] > 0]
    
    s.convergence=convergence_curve
    s.optimizer="WOA"   
    s.objfname=objf.__name__
    s.bestfitness = Leader_score
    s.bestf1 = Leader_f1
    s.bestIndividual = feature_index
    return s

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
X_train=np.load("/content/drive/My Drive/Major2/X_train.npy")
y_train=np.load("/content/drive/My Drive/Major2/y_train.npy")
X_test=np.load("/content/drive/My Drive/Major2/X_test.npy")
y_test=np.load("/content/drive/My Drive/Major2/y_test.npy")

In [0]:
X_train = X_train.reshape(840, 208*176)
X_test = X_test.reshape(360, 208*176)

In [0]:
X_train.shape

(840, 36608)

In [0]:
sol = WOA(X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test,Max_iter=50)

In [0]:
print(sol.bestIndividual)

[47, 147, 198, 237, 291, 315, 368, 371, 372, 379, 427, 610, 895, 910, 925, 928, 953, 975, 1007, 1037, 1050, 1060, 1073, 1115, 1165, 1215, 1232, 1289, 1360, 1390, 1432, 1433, 1445, 1532, 1662, 1696, 1700, 1796, 1809, 1826, 1847, 1872, 1939, 1956, 2011, 2160, 2162, 2173, 2315, 2361, 2622, 2820, 2919, 2929, 3010, 3018, 3043, 3079, 3085, 3135, 3169, 3336, 3383, 3390, 3571, 3610, 3623, 3679, 3726, 3846, 3988, 3992, 4075, 4086, 4214, 4413, 4414, 4678, 4819, 4853, 4855, 4864, 4930, 5193, 5240, 5289, 5445, 5704, 5798, 5931, 5935, 5942, 5986, 6037, 6050, 6089, 6092, 6280, 6287, 6300, 6488, 6541, 6570, 6575, 6594, 6608, 6623, 6634, 6653, 6670, 6720, 6768, 6839, 6851, 6870, 7185, 7206, 7352, 7493, 7497, 7500, 7513, 7520, 7548, 7654, 7699, 7727, 7856, 7882, 7920, 7935, 7949, 8124, 8140, 8261, 8272, 8354, 8414, 8444, 8478, 8507, 8578, 8722, 8738, 8744, 8754, 8931, 8939, 9011, 9031, 9118, 9152, 9185, 9255, 9286, 9370, 9380, 9455, 9487, 9525, 9555, 9585, 9601, 9753, 9759, 9940, 10051, 10150, 10165, 1

In [0]:
indices=sol.bestIndividual

In [0]:
print(len(sol.bestIndividual))

567


In [0]:
X_train_WOA=X_train[:,indices]
X_test_WOA=X_test[:,indices]

In [0]:
X_train_WOA.shape

(840, 567)

In [0]:
np.save("X_train_WOA.npy",X_train_WOA)
np.save("y_train_WOA.npy",y_train)
np.save("X_test_WOA.npy",X_test_WOA)
np.save("y_test_WOA.npy",y_test)

In [0]:
from google.colab import files
files.download('X_train_WOA.npy')
files.download('X_test_WOA.npy')
files.download('y_train_WOA.npy')
files.download('y_test_WOA.npy')